In [80]:
import sklearn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack, coo_matrix, vstack


In [30]:
pip install scikit-multilearn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.4/89.4 kB 1.6 MB/s eta 0:00:00


In [76]:
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from skmultilearn.adapt import MLkNN
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier

from sklearn.multioutput import ClassifierChain
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import MultiLabelBinarizer
from scipy.sparse import coo_matrix

In [63]:
file_df = "/content/drive/MyDrive/nlp/classification/df_clean.csv"
df = pd.read_csv(file_df)
df['clean_body'] = df['clean_body'].astype(str)
df['clean_title'] = df['clean_title'].astype(str)
df['clean_tag'] = df['clean_tag'].astype(str)

df_g = df.groupby(['clean_title', 'clean_body'])['clean_tag'].apply(lambda x: ','.join(x)).reset_index()
df_g['clean_tag'] = df_g['clean_tag'].apply(lambda tags: ','.join(set(tags.split(','))))

all_tags = ','.join(df_g['clean_tag']).split(',')
tag_counts = pd.Series(all_tags).value_counts()
top_2_tags = tag_counts.nlargest(2).index.to_list()
def select_top_2_tags(tags):
  tag_list = tags.split(',')
  top_2_tags_entry = sorted(tag_list, key = lambda x: tag_counts.get(x, 0), reverse = True)[:2]
  return ','.join(top_2_tags_entry)

df_g['clean_tag'] = df_g['clean_tag'].apply(select_top_2_tags)

In [64]:
df = df_g.head(5000)
df.shape

(5000, 3)

# Basic model

In [66]:
# Text vectorization using TF-IDF for Text1
X_title = df['clean_title']
X_body = df['clean_body']
vectorizer_title = TfidfVectorizer(max_features=100, stop_words='english')
X_title_vec = vectorizer_title.fit_transform(X_title)

# Text vectorization using TF-IDF for Text2
vectorizer_body = TfidfVectorizer(max_features=100, stop_words='english')
X_body_vec = vectorizer_body.fit_transform(X_body)

In [67]:
df['clean_tag'] = df['clean_tag'].apply(lambda x: ','.join(x.split(',') + ['dummy_tag']) if len(x.split(',')) == 1 else ','.join(x.split(',')))
df['clean_tag'] = df['clean_tag'].apply(lambda x: ','.join(x.split(',') + ['dummy_tag1', 'dummy_tag']) if len(x.split(',')) == 0 else ','.join(x.split(',')))

<ipython-input-67-f64dc7a0b908>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['clean_tag'] = df['clean_tag'].apply(lambda x: ','.join(x.split(',') + ['dummy_tag']) if len(x.split(',')) == 1 else ','.join(x.split(',')))
<ipython-input-67-f64dc7a0b908>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['clean_tag'] = df['clean_tag'].apply(lambda x: ','.join(x.split(',') + ['dummy_tag1', 'dummy_tag']) if len(x.split(',')) == 0 else ','.join(x.split(',')))


In [69]:
mlb = MultiLabelBinarizer()
#y = mlb.fit_transform(df['clean_tag'].apply(lambda x: x.split(',') if (len(x.split(',') <2) else x.split(',').append('tech')))
y = mlb.fit_transform(df['clean_tag'].apply(lambda x: x.split(',')))

In [70]:
y

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [75]:
# Print the shape of the target variable
print("Shape of y:", y.shape)

# Print the unique classes in the binarized target variable
print("Unique Classes in y:", mlb.classes_)

Shape of y: (5000, 21)
Unique Classes in y: ['array' 'c' 'csv' 'dictionary' 'django' 'dummy_tag' 'flask' 'google'
 'html' 'list' 'loop' 'matplotlib' 'mysql' 'numpy' 'panda' 'pyqt' 'python'
 'regex' 'string' 'tkinter' 'web']


In [71]:
weight_title = 0.6
weight_body = 0.4

# weighted sum
X_combined= hstack([weight_title * X_title_vec , weight_body * X_body_vec])

In [72]:
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)

In [89]:
# classifier = ClassifierChain(LogisticRegression())
# classifier.fit(X_train, y_train)
# predictions = classifier.predict(X_test)

In [88]:
# # MultiOutputClassifier with Logistic Regression
# classifier = MultiOutputClassifier(LogisticRegression())
# classifier.fit(X_train, y_train)
# predictions = classifier.predict(X_test)

In [77]:
classifiers = [OneVsRestClassifier(LogisticRegression()) for _ in range(y.shape[1])]
for i, classifier in enumerate(classifiers):
    classifier.fit(X_train, y_train[:, i])

/usr/local/lib/python3.10/dist-packages/sklearn/multiclass.py:77: UserWarning: Label not 1 is present in all training examples.
  warnings.warn(


In [81]:
predictions = [classifier.predict(X_test) for classifier in classifiers]

# Combine predictions
combined_predictions = np.column_stack(predictions)

accuracy = accuracy_score(y_test, combined_predictions)
print(f"Accuracy: {accuracy}")

Accuracy: 0.673


In [86]:
#predictions[0]
predictions_proba = [classifier.predict_proba(X_test) for classifier in classifiers]

In [87]:
predictions_proba[0]

array([[0.98316895, 0.01683105],
       [0.99721794, 0.00278206],
       [0.99656365, 0.00343635],
       ...,
       [0.99779008, 0.00220992],
       [0.99729422, 0.00270578],
       [0.99640692, 0.00359308]])

# models with transformers

In [90]:
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset
import torch

In [91]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tokenized_inputs = tokenizer(df['clean_title'].tolist(), df['clean_body'].tolist(), padding=True, truncation=True, return_tensors='pt')

# Combine the two inputs (clean_title and clean_body) into a single input tensor
input_ids = tokenized_inputs['input_ids']
attention_mask = tokenized_inputs['attention_mask']

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

In [92]:
X_train, X_test, y_train, y_test = train_test_split(input_ids, y, test_size=0.2, random_state=42)

In [93]:
train_data = TensorDataset(X_train, torch.tensor(y_train, dtype=torch.float32))
test_data = TensorDataset(X_test, torch.tensor(y_test, dtype=torch.float32))
train_loader = DataLoader(train_data, batch_size=8, shuffle=True)
test_loader = DataLoader(test_data, batch_size=8)

In [94]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(mlb.classes_))

optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
num_epochs = 1

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.train()
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs, labels=labels)
        #outputs = model(inputs, attention_mask=attention_mask[inputs.indices], labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
